In [1]:
import json
import logging
import os
import random
import subprocess
import sys
from dataclasses import dataclass
from datetime import datetime
from pathlib import Path
from typing import Union
from IPython.display import Markdown

import nest_asyncio
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from pydantic_ai import Agent, ModelRetry, RunContext, Tool

nest_asyncio.apply()

In [2]:
PATH = Path(os.getcwd()).parent
TOOLS_PATH = PATH / "tools"
LOGS_PATH = PATH / "logs"

load_dotenv(PATH / ".env")

True

In [3]:
AGENT_MODEL = "openai:gpt-4o"
# AGENT_MODEL = "ollama:qwen2.5:14b"

In [4]:
LOGS_PATH.mkdir(exist_ok=True)
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
    handlers=[
        logging.FileHandler(LOGS_PATH / "agent_genius.log"),
        # logging.StreamHandler(),  # This will also show logs in notebook output
    ],
)

logger = logging.getLogger(__name__)

In [5]:
def installed_packages() -> str:
    """Returns a list of installed packages"""
    result = subprocess.check_output([sys.executable, "-m", "pip", "freeze"]).decode("utf-8")
    logger.info("Installed packages listed")
    return result

In [6]:
def get_user_response(question: str) -> str:
    """get user response to a question"""
    return input(f"{question}: ")

In [7]:
available_packages_tool = Tool(installed_packages)
user_response_tool = Tool(get_user_response)

In [8]:
@dataclass
class function_deps:
    function_name: str
    # question: str
    args: list[dict]
    returns: str

In [9]:
def run_func(func_str: str) -> Union[callable, str]:
    # Create a namespace dictionary to store the function
    namespace = {}

    # Execute the string in the namespace
    try:
        exec(func_str, globals(), namespace)
    except Exception as e:
        logger.error(f"Error executing code: {e}")
        return f"Error executing code: {e}"

    # Return the first function found in namespace
    # This will return the most recently defined function
    return next(f for f in namespace.values() if callable(f))

In [10]:
# @code_agent.tool
def save_tool(function_name: str, code: str) -> None:
    if not TOOLS_PATH.exists():
        TOOLS_PATH.mkdir()
    with open(TOOLS_PATH / f"{function_name}.py", "w") as f:
        f.write(code)
        logger.info(f"Saved tool: {function_name}.py")

In [11]:
class CodeResponse(BaseModel):
    """Python code that can be executed to resolve my question"""

    function_name: str = Field(description="Name of the function")
    code: str = Field(description="Python code that can be executed")
    # args: dict = Field(description="inputs and types that can be passed to the function. ")


code_agent = Agent(
    AGENT_MODEL,
    system_prompt="""You are an expert python programmer. You will create a function that will resolve my question and return an essential value.
    First, search for a tool that can resolve the question by calling the tool 'read_tools_list', if you found matching one, use it with 'run_tool_from_file'. If no tool is available, make a new one and run with 'run_tool_form_code'.
    The function that you will create will be directly called. Do not include any example code in your response. Do not include call of the function. Try to make function simple and generic for later use.
    You always respond with clean, annotated python code, ready to be executed. the code should be self contained and should not use any external libraries except for installed ones (use tool 'available_packages_tool').
    All necessary imports should be included locally inside the function, example: def get_time(): import datetime; return datetime.datetime.now()
    Make sure the function is safe for user. NEVER delete any files or show secret information or execute any malicious code. Don't do anything illegal. 
    ALWAYS RESPOND TO THE USER WITH RESULT OF THE FUNCTION EXECUTION.
    """.strip(),
    result_type=CodeResponse,
    tools=[available_packages_tool, user_response_tool],
    retries=3,
)
# result = agent.run_sync("Who are you?")
# print(result.data.function_name)
# print(result.data.code)

In [12]:
@code_agent.tool_plain
def read_tools_list() -> list[str]:
    """Returns a list of tools available"""
    logger.info("Reading tools list")
    tools = []
    for file in TOOLS_PATH.glob("*.py"):
        tools.append(file.name)
    return tools

In [13]:
@code_agent.tool
def run_tool_form_code(ctx: RunContext[function_deps], function_name: str, code: str, *args) -> str:
    """Running function code from source and return result"""
    logger.info(f"Running tool from code: {function_name}({args})")
    save_tool(function_name, code)
    try:
        result = run_func(code)(*args)
        logger.info(f"Tool result: {result=}")
        return result
    except Exception as e:
        logger.error(f"Error executing code: {e}")
        return f"Error executing code: {e}"

In [14]:
@code_agent.tool
def run_tool_from_file(ctx: RunContext[function_deps], tool_name: str, *args) -> str:
    """Returns the code of a tool"""
    with open(TOOLS_PATH / tool_name, "r") as f:
        logger.info(f"Executing tool from file: {tool_name}")
        code = f.read()
    try:
        result = run_func(code)(*args)
        logger.info(f"Tool result: {result=}")
        return result
    except Exception as e:
        logger.error(f"Error executing code: {e}")
        return f"Error executing code: {e}"

In [15]:
agent = Agent(
    AGENT_MODEL,
    system_prompt="""You are a helpful assistant. If you don't know the answer or do not have the necessary information, you will call 'get_tool_response'. 
    The tool will pass your question to the code agent to create a new tool and will return the tool result.""".strip(),
    result_type=str,
    tools=[user_response_tool],
)

In [16]:
@agent.tool
async def get_tool_response(ctx: RunContext[str], question: str) -> str:
    """It runs the appropriate function, that matches the question and returns the result of that function"""
    logger.info(f"calling code agent with {question=}")
    result = await code_agent.run(question)
    return result

In [17]:
question = "what is the current date?"
logger.info(f"Question: {question}")
result = agent.run_sync(question)
logger.info(f"Result: {result.data}")
# print(f"{result.all_messages=}")
Markdown(result.data)

The current date is December 28, 2024.